In [1]:
# corrected_multi_agent.py
from typing import List, Dict, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.graph import add_messages
from typing import Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from datetime import datetime

# ---- LLM (your existing LLM)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

# ---- State
class SupervisorState(MessagesState):
    # attach messages using add_messages helper so the graph can pass messages in
    messages: Annotated[List[BaseMessage], add_messages] = []
    next_agent: str = "supervisor"
    Images_data: str = ""
    scientific_data: str = ""
    products_data: str = ""
    final_answer: str = ""
    task_complete: bool = False
    current_task: str = ""

# ---- Helper: normalized mapping from supervisor token -> node name
SUPERVISOR_TO_NODE = {
    "image_analyzer": "image_analysis_agent",
    "scientific_researcher": "scientific_data_agent",
    "products_expert": "products_data_agent",
    "done": "final_answer_agent",
}

# ---- Supervisor (robust)
def supervisor_agent(state: SupervisorState) -> Dict:
    messages = state.get("messages", [])
    task = messages[-1].content if messages else ""

    # flags
    has_image = bool(state.get("Images_data"))
    has_scientific = bool(state.get("scientific_data"))
    has_product_info = bool(state.get("products_data"))
    has_final_answer = bool(state.get("final_answer"))

    # Build a short, strict prompt (ask LLM to return exactly one token)
    system = SystemMessage(
        content=(
            "You are a supervisor. Decide which single agent to run next. "
            "Respond with exactly one of: image_analyzer / scientific_researcher / products_expert / DONE.\n"
            "Interpret these booleans: has_image, has_scientific, has_product_info, has_final_answer."
        )
    )

    human = HumanMessage(
        content=(
            f"Task: {task}\n"
            f"has_image: {has_image}\n"
            f"has_scientific: {has_scientific}\n"
            f"has_product_info: {has_product_info}\n"
            f"has_final_answer: {has_final_answer}\n\n"
            "Which agent next? (one word from image_analyzer / scientific_researcher / products_expert / DONE)"
        )
    )

    decision = llm.invoke([system, human])
    decision_text = decision.content.strip().lower()
    print("Supervisor LLM raw decision:", repr(decision_text))

    # sanitize
    choice = None
    for token in SUPERVISOR_TO_NODE.keys():
        if token in decision_text:
            choice = token
            break
    if not choice:
        # fallback heuristic
        if not has_image:
            choice = "image_analyzer"
        elif not has_scientific:
            choice = "scientific_researcher"
        elif not has_product_info:
            choice = "products_expert"
        else:
            choice = "done"

    next_node = SUPERVISOR_TO_NODE[choice]

    # human-friendly message to store in messages
    mapping_msg = {
        "image_analyzer": "📋 Supervisor: Assign to image analysis.",
        "scientific_researcher": "📋 Supervisor: Assign to scientific research.",
        "products_expert": "📋 Supervisor: Assign to product expert.",
        "done": "✅ Supervisor: All tasks complete, preparing final answer."
    }

    return {
        "messages": [AIMessage(content=mapping_msg[choice])],
        "next_agent": next_node,
        "current_task": task
    }

# ---- Image analysis agent (placeholder implementation)
def image_analysis_agent(state: SupervisorState) -> Dict:
    # Use the provided image path in state.current_task or Images_data; here we simulate an LLM analysis.
    # In your production code you should replace this with proper image uploading + vision model call.
    image_path = state.get("Images_data", "") or "test.png"

    prompt = (
        f"You are an expert in hair care. We have an image at path: {image_path}.\n"
        "Based on that image, give a single-line hair type label (choose short) and a 1-2 sentence observation."
    )
    resp = llm.invoke([HumanMessage(content=prompt)])
    analysis_text = resp.content.strip()

    state_update = {
        "messages": [AIMessage(content=f"🖼️ Image Analysis Agent: {analysis_text}")],
        "Images_data": analysis_text,
        "next_agent": "supervisor"
    }
    print("Image agent output:", analysis_text)
    return state_update

# ---- Scientific data agent (placeholder - use your retriever here)
def scientific_data_agent(state: SupervisorState) -> Dict:
    task = state.get("current_task", "general haircare")
    prompt = (
        f"As a scientific researcher summarize top 3 evidence-backed points about: {task}.\n"
        "Return a short paragraph and list 1-2 citations (if none available say 'no citations')."
    )
    resp = llm.invoke([HumanMessage(content=prompt)])
    doc_text = resp.content.strip()

    return {
        "messages": [AIMessage(content=f"🔬 Scientific Agent: Retrieved data.")],
        "scientific_data": doc_text,
        "next_agent": "supervisor"
    }

# ---- Products data agent (your product catalog)
def products_data_agent(state: SupervisorState) -> Dict:
    # You can replace with real DB/API lookup. This is a safe placeholder returning a small product list.
    products_data = (
        "Gliss Ultimate Repair - Liquid Keratin: for very damaged hair.\n"
        "Gliss Aqua Revive - Hyaluron Complex: for dry hair hydration.\n"
        "Gliss Oil Nutritive - Marula Oil: for frizzy long hair."
    )

    return {
        "messages": [AIMessage(content="🛍️ Products Agent: Provided product recommendations.")],
        "products_data": products_data,
        "next_agent": "supervisor"
    }

# ---- Final answer agent (aggregates and produces final answer)
def final_answer_agent(state: SupervisorState) -> Dict:
    images_data = state.get("Images_data", "No image data available.")
    scientific_data = state.get("scientific_data", "No scientific data available.")
    products_data = state.get("products_data", "No product data available.")
    task = state.get("current_task", "the task")

    prompt = (
        f"Create a short user-friendly answer for: {task}\n\n"
        f"Image analysis:\n{images_data}\n\n"
        f"Scientific summary:\n{scientific_data}\n\n"
        f"Product suggestions:\n{products_data}\n\n"
        "Return a 5-8 sentence final recommendation and a 1-line actionable tip."
    )
    resp = llm.invoke([HumanMessage(content=prompt)])
    final = resp.content.strip()

    return {
        "messages": [AIMessage(content=f"✅ Final Answer Agent: {final}")],
        "final_answer": final,
        "task_complete": True,
        "next_agent": "end"
    }

# ---- Router (maps the next_agent string to nodes)
def router(state: SupervisorState) -> Literal["supervisor", "image_analysis_agent", "scientific_data_agent", "products_data_agent", "final_answer_agent", "__end__"]:
    next_agent = state.get("next_agent", "supervisor")
    if next_agent == "end" or state.get("task_complete", False):
        return END

    if next_agent in ["supervisor", "image_analysis_agent", "scientific_data_agent", "products_data_agent", "final_answer_agent"]:
        return next_agent
    return "supervisor"

# ---- Build workflow
workflow = StateGraph(SupervisorState)
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("image_analysis_agent", image_analysis_agent)
workflow.add_node("scientific_data_agent", scientific_data_agent)
workflow.add_node("products_data_agent", products_data_agent)
workflow.add_node("final_answer_agent", final_answer_agent)
workflow.set_entry_point("supervisor")

for node in ["supervisor", "image_analysis_agent", "scientific_data_agent", "products_data_agent", "final_answer_agent"]:
    workflow.add_conditional_edges(
        node,
        router,
        {
            "supervisor": "supervisor",
            "image_analysis_agent": "image_analysis_agent",
            "scientific_data_agent": "scientific_data_agent",
            "products_data_agent": "products_data_agent",
            "final_answer_agent": "final_answer_agent",
            END: END
        }
    )

graph = workflow.compile()

# ---- Example run: pass an initial HumanMessage with an actual task
if __name__ == "__main__":
    # Provide an initial task message so supervisor has something to act on
    initial = HumanMessage(content="Assess hair condition from an uploaded image and recommend a product for dry, damaged long hair.")
    response = graph.invoke(initial)
    print("GRAPH RESPONSE KEYS:", response.keys())
    print("FINAL ANSWER (if present):\n", response.get("final_answer"))


Supervisor LLM raw decision: 'image_analyzer'
Image agent output: As an AI, I am unable to view or process images from local file paths like "test.png." Therefore, I cannot provide a hair type label or observation based on an image I cannot see.

If you can describe the hair in the image, I would be happy to provide an assessment!
Supervisor LLM raw decision: 'done'
GRAPH RESPONSE KEYS: dict_keys(['messages', 'next_agent', 'Images_data', 'final_answer', 'task_complete', 'current_task'])
FINAL ANSWER (if present):
 It appears I wasn't able to process the image you provided, as I cannot directly view files from your local computer. As an AI, my capabilities are limited to text-based input, so I can't 'see' images like a human can. To help me understand the hair type, please describe it in detail. You could mention its texture (straight, wavy, curly, coily), thickness (fine, medium, thick), and any specific characteristics like frizz, oiliness, or dryness. Once you provide a description, 